# 06-10-02

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Transition Matrices, HighLevel Features)
    - MinMax Scaling
    - Remove Constant Features
    - Euclidean Distance
- Categorical Features
    - Hot Encoding
    - Jaccard

In [36]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf 
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [37]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [38]:
cover = loadCoverGroups()
listCovers = cover[12]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [39]:
coverSongId

6001997

In [40]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [41]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [42]:
df = pd.merge(df,transitionMatrices, on='id')

In [43]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [44]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2,category
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
16,836,0.511,0.517,0.0328,0.130000,0.0740,0.433,75.670,0.000000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
17,1334,0.330,0.905,0.0497,0.000233,0.0712,0.259,163.142,0.104478,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
22,2147,0.311,0.815,0.0443,0.253000,0.6080,0.700,147.173,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,6002494,0.713,0.798,0.2190,0.366000,0.2460,0.721,176.073,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [45]:
df = df.drop(columns=['category'])

In [46]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,23,0.494,0.756,0.0432,0.001400,0.1640,0.589,133.337,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,120,0.319,0.807,0.0388,0.001040,0.1310,0.601,118.097,0.029412,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
16,836,0.511,0.517,0.0328,0.130000,0.0740,0.433,75.670,0.000000,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
17,1334,0.330,0.905,0.0497,0.000233,0.0712,0.259,163.142,0.104478,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
22,2147,0.311,0.815,0.0443,0.253000,0.6080,0.700,147.173,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5970,6002494,0.713,0.798,0.2190,0.366000,0.2460,0.721,176.073,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
5973,6002497,0.725,0.583,0.0427,0.300000,0.0665,0.762,79.984,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5974,6002498,0.399,0.787,0.0499,0.019700,0.0685,0.572,117.089,0.000000,0.015385,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5975,6002499,0.376,0.435,0.0264,0.513000,0.1380,0.304,80.002,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [48]:
ids[ids == originalSongId]

2036    6001918
Name: id, dtype: int64

## Pipeline

In [49]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [50]:
dfEuclidean

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,997,998,999,acousticness,danceability,energy,liveness,speechiness,tempo,valence
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.001400,0.494,0.756,0.1640,0.0432,133.337,0.589
1,0.029412,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.001040,0.319,0.807,0.1310,0.0388,118.097,0.601
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.130000,0.511,0.517,0.0740,0.0328,75.670,0.433
3,0.104478,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000233,0.330,0.905,0.0712,0.0497,163.142,0.259
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.115942,0.253000,0.311,0.815,0.6080,0.0443,147.173,0.700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.366000,0.713,0.798,0.2460,0.2190,176.073,0.721
2421,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.300000,0.725,0.583,0.0665,0.0427,79.984,0.762
2422,0.000000,0.015385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.019700,0.399,0.787,0.0685,0.0499,117.089,0.572
2423,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.513000,0.376,0.435,0.1380,0.0264,80.002,0.304


In [51]:
dfJaccard

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2421,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2422,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2423,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
# dfEuclidean = applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
dfEuclidean = dfEuclidean.copy()
dfEuclidean = removeConstantFeatures(dfEuclidean)
# dfEuclidean= applyBoxCoxFeatures(dfEuclidean, ['speechiness', 'acousticness', 'liveness', 'valence'])
# dfEuclidean= applyBoxCoxAllFeatures(dfEuclidean)
# dfEuclidean = centerAndScale(dfEuclidean)

In [53]:
scaler = MinMaxScaler()
scaler.fit(dfEuclidean)
dfEuclidean = pd.DataFrame(scaler.transform(dfEuclidean)).copy()

### Original Song & Cover Song

In [54]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfEuclidean)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfEuclidean)

### Get Rankings Euclidean

In [55]:
ranksDFEuclidean  = applyEuclideanDistance(dfEuclidean, ids, originalSongVector)
ranksDFEuclidean

distance


,id,rank
0,6001918,0.000000
1,6001574,0.848654
2,6000373,0.984102
3,6001512,0.994384
4,6001270,0.997077
...,...,...
2420,6002278,3.476467
2421,6000847,3.522949
2422,6000959,3.794594
2423,6002260,3.797455


In [56]:
ranksDFEuclidean[ranksDFEuclidean.id == originalSongId]

,id,rank
0,6001918,0.0


In [57]:
ranksDFEuclidean[ranksDFEuclidean['id']==coverSongId]

,id,rank
1376,6001997,1.940265


In [58]:
ranksDFEuclidean.loc[ranksDFEuclidean.id.isin(listCovers)]

,id,rank
0,6001918,0.000000
1376,6001997,1.940265


### Get Rankings Jaccard

In [59]:
originalSongIndex,originalSongVector = getIndexAndVector(originalSongId, ids, dfJaccard)
coverSongIndex,coverSongVector = getIndexAndVector(coverSongId, ids, dfJaccard)

In [60]:
ranksDFJaccard  = applyJaccard(dfJaccard, ids, originalSongVector)

In [61]:
ranksDFJaccard

,id,rank
0,6002474,0.0
1,1863781,0.0
2,6001901,0.0
3,6002056,0.0
4,6001718,0.0
...,...,...
2420,6000196,1.0
2421,6000198,1.0
2422,6000767,1.0
2423,390605,1.0


In [62]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
9,6001997,0.333333


In [63]:
ranksDFJaccard[ranksDFJaccard['id']==coverSongId]

,id,rank
9,6001997,0.333333


In [64]:
ranksDFJaccard.loc[ranksDFJaccard.id.isin(listCovers)]

,id,rank
7,6001918,0.000000
9,6001997,0.333333


### Experiment to unify ranks

In [65]:
ranks = pd.merge(ranksDFJaccard,ranksDFEuclidean, on='id')

In [66]:
ranks['new_rank'] = ranks['rank_x'] + ranks['rank_y']

In [67]:
ranks = ranks.sort_values(by='new_rank')
ranks.reset_index(drop=True, inplace=True)

In [68]:
ranks.head(10)

,id,rank_x,rank_y,new_rank
0,6001918,0.000000,0.000000,0.000000
1,6001574,0.571429,0.848654,1.420082
2,6000754,0.333333,1.146016,1.479349
3,6001868,0.333333,1.250850,1.584183
4,6002074,0.571429,1.030759,1.602188
5,6002017,0.571429,1.120548,1.691977
6,6002056,0.000000,1.704581,1.704581
7,6001788,0.571429,1.134018,1.705446
8,6002002,0.571429,1.143903,1.715332
9,6000373,0.750000,0.984102,1.734102


In [69]:
ranks[ranks['id']==coverSongId]

,id,rank_x,rank_y,new_rank
237,6001997,0.333333,1.940265,2.273598


In [70]:
ranks.loc[ranks.id.isin(listCovers)]

,id,rank_x,rank_y,new_rank
0,6001918,0.000000,0.000000,0.000000
237,6001997,0.333333,1.940265,2.273598
